# FIFA 선수 이적료 예측 경진대회

### FIFA_train.csv / FIFA_test.csv
id : 선수 고유의 아이디\
name : 이름\
age : 나이\
continent : 선수들의 국적이 포함되어 있는 대륙입니다\
contract_until : 선수의 계약기간이 언제까지인지 나타내어 줍니다\
position : 선수가 선호하는 포지션입니다. ex) 공격수, 수비수 등\
prefer_foot : 선수가 선호하는 발입니다. ex) 오른발\
reputation : 선수가 유명한 정도입니다. ex) 높은 수치일 수록 유명한 선수\
stat_overall : 선수의 현재 능력치 입니다.\
stat_potential : 선수가 경험 및 노력을 통해 발전할 수 있는 정도입니다.\
stat_skill_moves : 선수의 개인기 능력치 입니다.\
value : FIFA가 선정한 선수의 이적 시장 가격 (단위 : 유로) 입니다\


### submission.csv (제출 파일 형식)
id : 선수 고유의 아이디\
value : 예측된 선수 이적 시장 가격을 기입


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

## 1.데이터 로드 및 정보 확인

1. 데이터 로드 및 라벨 지정

In [2]:
train = pd.read_csv("./FIFA_train.csv")
label = train['value']
test = pd.read_csv("./FIFA_test.csv")
submission = pd.read_csv("./submission.csv")

2. 데이터 정보 확인

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8932 non-null   int64  
 1   name              8932 non-null   object 
 2   age               8932 non-null   int64  
 3   continent         8932 non-null   object 
 4   contract_until    8932 non-null   object 
 5   position          8932 non-null   object 
 6   prefer_foot       8932 non-null   object 
 7   reputation        8932 non-null   float64
 8   stat_overall      8932 non-null   int64  
 9   stat_potential    8932 non-null   int64  
 10  stat_skill_moves  8932 non-null   float64
 11  value             8932 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 837.5+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3828 entries, 0 to 3827
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3828 non-null   int64  
 1   name              3828 non-null   object 
 2   age               3828 non-null   int64  
 3   continent         3828 non-null   object 
 4   contract_until    3828 non-null   object 
 5   position          3828 non-null   object 
 6   prefer_foot       3828 non-null   object 
 7   reputation        3828 non-null   float64
 8   stat_overall      3828 non-null   int64  
 9   stat_potential    3828 non-null   int64  
 10  stat_skill_moves  3828 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 329.1+ KB


3. null 값 확인

In [5]:
train.isnull().sum()

id                  0
name                0
age                 0
continent           0
contract_until      0
position            0
prefer_foot         0
reputation          0
stat_overall        0
stat_potential      0
stat_skill_moves    0
value               0
dtype: int64

## 2.데이터 전처리

1. 필요없는 데이터 삭제(라벨은 앞에서 지정하였으므로 불필요)

In [6]:
train.drop(['id', 'name', 'value'], axis=1, inplace=True)
test.drop(['id', 'name'], axis=1, inplace=True)

In [8]:
train.shape, test.shape

((8932, 9), (3828, 9))

2. Categorical, Numerical Columns 각각 분리해 준다.

In [9]:
cal_c = test.dtypes[test.dtypes == "object"].index.values
num_c = test.dtypes[test.dtypes != "object"].index.values
print(f'Categorical Columns: \n {cal_c}')
print(f'Numerical Columns: \n {num_c}')

Categorical Columns: 
 ['continent' 'contract_until' 'position' 'prefer_foot']
Numerical Columns: 
 ['age' 'reputation' 'stat_overall' 'stat_potential' 'stat_skill_moves']


3. contract_utill Column에서 연도만 추출
* apply 메서드는 DataFrame에 함수를 적용하여 반환합니다.

In [10]:
train.contract_until.value_counts()

2019            2366
2021            2308
2020            2041
2022             761
2023             506
Jun 30, 2019     501
2018             327
Dec 31, 2018      64
May 31, 2019      19
2024              12
Jan 31, 2019      10
Jun 30, 2020       9
2025               3
Jan 1, 2019        2
2026               1
May 31, 2020       1
Jan 12, 2019       1
Name: contract_until, dtype: int64

In [11]:
def func(string:object):
    string = string[-4:] # 뒤에서 네번째 까지(연도))
    return int(string)

train['contract_until'] = train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

In [12]:
train.contract_until.value_counts()

2019    2899
2021    2308
2020    2051
2022     761
2023     506
2018     391
2024      12
2025       3
2026       1
Name: contract_until, dtype: int64

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(train, label, test_size=0.2, random_state=2022)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((7145, 9), (1787, 9), (7145,), (1787,))

## 3.Scailing

In [18]:
sdscaler = StandardScaler()
sdscaled_train = pd.DataFrame(sdscaler.fit_transform(X_train, columns = X_train[num_c])

SyntaxError: unexpected EOF while parsing (3629944023.py, line 2)

In [ ]:
sdscaled_train

## 4.Modeling

In [ ]:
dt_regr = DecisionTreeRegressor(max_depth=4)

In [ ]:
dt_regr.fit(X_train['stat_overall'].values.reshape(-1,1), y_train)

In [ ]:
y_pred = dt_regr.predict(X_test['stat_overall'].values.reshape(-1,1))

In [ ]:
print('단순 결정 트리 회귀 R2 : {:.4f}'.format(r2_score(y_test, y_pred))) # depth = 5 

In [ ]:
arr = np.arange(1,10)

In [ ]:
best_depth = 0
best_r2 = 0

for depth in arr:
  dt_regr = DecisionTreeRegressor(max_depth=depth)
  dt_regr.fit(X_train['stat_overall'].values.reshape(-1,1), y_train)
  y_pred = dt_regr.predict(X_test['stat_overall'].values.reshape(-1,1))
  
  temp_r2 = r2_score(y_test, y_pred)
  print('\n단순 결정 트리 회귀 depth ={} R2 : {:.4f}'.format(depth, temp_r2))

  if best_r2 < temp_r2:
    best_depth = depth
    best_r2 = temp_r2

print('최적의 결과는 depth={} r2={:.4f}'.format(best_depth, best_r2))

In [ ]:
dt_regr = DecisionTreeRegressor(max_depth=5)

In [ ]:
dt_regr.fit(X_train, y_train)

In [ ]:
y_pred = dt_regr.predict(X_test)

In [ ]:
print('다중 결정 트리 회귀 R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

In [ ]:
best_depth = 0
best_r2 = 0

for depth in arr:
  dt_regr = DecisionTreeRegressor(max_depth=depth)
  dt_regr.fit(X_train, y_train)
  y_pred = dt_regr.predict(X_test)
  
  temp_r2 = r2_score(y_test, y_pred)
  print('\n다중 결정 트리 회귀 depth ={} R2 : {:.4f}'.format(depth, temp_r2))

  if best_r2 < temp_r2:
    best_depth = depth
    best_r2 = temp_r2

print('최적의 결과는 depth={} r2={:.4f}'.format(best_depth, best_r2))

,age,reputation,stat_overall,stat_potential,stat_skill_moves
2134,30,1.0,72,72,1.0
6041,22,1.0,64,72,2.0
7241,20,1.0,61,72,2.0
2033,26,1.0,72,75,3.0
5959,21,1.0,64,74,3.0
...,...,...,...,...,...
6384,19,1.0,63,71,3.0
4720,25,1.0,67,71,3.0
173,29,3.0,82,82,4.0
1244,32,1.0,74,74,3.0
